In [1]:
import tensorflow as tf
import numpy as np

/Users/aaron/Documents/ml/kaggle/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.5 # Dropout, probability to keep units

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input]) # input, i.e. pixels that constitute the image
y = tf.placeholder(tf.float32, [None, n_classes]) # labels, i.e which digit the image is
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [28]:
# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    # Fully connected layer
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

# Store layers weight &amp; bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = conv_net(x, weights, biases, keep_prob)

In [9]:
pred

<tf.Tensor 'Add_1:0' shape=(?, 10) dtype=float32>

In [16]:
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [17]:
image = mnist.train.images[0]
image.shape

(784,)

In [19]:
# sess = tf.InteractiveSession()

In [21]:
image_2d = tf.reshape(image, shape=[-1, 28, 28, 1])
image_2d

<tf.Tensor 'Reshape_4:0' shape=(1, 28, 28, 1) dtype=float32>

In [23]:
tf.nn.conv2d?

In [34]:
# 1 conv layer
x_train = tf.reshape(mnist.train.images, shape=[-1, 28, 28, 1])
window = tf.Variable(tf.random_normal([3, 3, 1, 32]))
strides = [1, 1, 1, 1]
padding = 'SAME'

conv1 = tf.nn.conv2d(input=x_train, filter=window, strides=strides, padding=padding)
conv1

<tf.Tensor 'Conv2D_5:0' shape=(55000, 28, 28, 32) dtype=float32>

In [35]:
# max pooling
k = 2
pool1 = tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')
pool1

<tf.Tensor 'MaxPool_4:0' shape=(?, 14, 14, 1) dtype=float32>

In [36]:
# 1 fc layer
num_classes = 10
fc1 = tf.reshape(pool1, shape=[-1, num_classes])
fc1

<tf.Tensor 'Reshape_12:0' shape=(?, 10) dtype=float32>

In [50]:
logits = fc1
labels = mnist.train.labels
loss = tf.losses.softmax_cross_entropy(onehot_labels=labels, logits=logits)
loss

<tf.Tensor 'softmax_cross_entropy_loss_1/value:0' shape=() dtype=float32>

In [55]:
# accuracy = tf.metrics.accuracy(
#         labels=labels, predictions=tf.argmax(logits, axis=1))
# accuracy

In [56]:
estimator = tf.estimator.EstimatorSpec(
        mode=tf.estimator.ModeKeys.EVAL,
        loss=loss)
estimator

EstimatorSpec(mode='eval', predictions={}, loss=<tf.Tensor 'softmax_cross_entropy_loss_1/value:0' shape=() dtype=float32>, train_op=None, eval_metric_ops={}, export_outputs=None, training_chief_hooks=(), training_hooks=(), scaffold=<tensorflow.python.training.monitored_session.Scaffold object at 0x1338db1d0>, evaluation_hooks=(), prediction_hooks=())

In [60]:
predicted_classes = tf.argmax(logits, 1)
predicted_classes

<tf.Tensor 'ArgMax_2:0' shape=(?,) dtype=int64>

In [61]:
accuracy = tf.metrics.accuracy(labels=labels,
                                   predictions=predicted_classes,
                                   name='acc_op')
accuracy

ValueError: Can not squeeze dim[1], expected a dimension of 1, got 10 for 'remove_squeezable_dimensions_1/Squeeze' (op: 'Squeeze') with input shapes: [55000,10].

In [52]:
labels.shape

(55000, 10)

In [53]:
logits.shape

TensorShape([Dimension(None), Dimension(10)])

In [29]:
fc1_vector_length = window.get_shape().as_list()[0]
fc1 = tf.reshape(conv1, [-1, fc1_vector_length])
fc1

<tf.Tensor 'Reshape_10:0' shape=(275968000, 5) dtype=float32>

In [30]:
window.shape

TensorShape([Dimension(5), Dimension(5), Dimension(1), Dimension(32)])

In [31]:
window.get_shape()

TensorShape([Dimension(5), Dimension(5), Dimension(1), Dimension(32)])

In [33]:
window.shape.as_list()

[5, 5, 1, 32]

In [27]:
# Fully connected layer
print(weights['wd1'])
print(biases['bd1'])

fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
fc1 = tf.nn.relu(fc1)

# # Apply Dropout
# fc1 = tf.nn.dropout(fc1, dropout)

# # Output, class prediction
# out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])

<tf.Variable 'Variable_2:0' shape=(3136, 1024) dtype=float32_ref>
<tf.Variable 'Variable_6:0' shape=(1024,) dtype=float32_ref>


ValueError: Dimensions must be equal, but are 5 and 3136 for 'MatMul_2' (op: 'MatMul') with input shapes: [275968000,5], [3136,1024].

In [5]:
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [6]:
def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [7]:
x = tf.reshape(x, shape=[-1, 28, 28, 1])

In [ ]:
x = tf.placeholder(tf.float32, [None, n_input]) # input, i.e. pixels that constitute the image